In [1]:
import pandas as pd

df = pd.read_csv('train_data.csv')

print(df)

        event_id  time_to_tca  mission_id       risk  max_risk_estimate  \
0              0     1.566798           5 -10.204955          -7.834756   
1              0     1.207494           5 -10.355758          -7.848937   
2              0     0.952193           5 -10.345631          -7.847406   
3              0     0.579669           5 -10.337809          -7.845880   
4              0     0.257806           5 -10.391260          -7.852942   
...          ...          ...         ...        ...                ...   
162629     13153     3.029751           1  -7.108630          -5.142668   
162630     13153     2.799253           1  -7.070070          -5.137869   
162631     13153     2.385399           1  -7.066209          -5.137034   
162632     13153     2.043339           1  -7.028307          -5.131297   
162633     13153     1.618095           1  -7.172372          -5.152181   

        max_risk_scaling  miss_distance  relative_speed  relative_position_r  \
0               8.6

In [14]:
df_filtered = df[df['time_to_tca'] > .5]
df_filtered_no_na = df_filtered.dropna()
df_filtered_relevant_columns = df_filtered_no_na[['event_id', 'risk', 'max_risk_scaling', 'max_risk_estimate', 'time_to_tca', 'mahalanobis_distance', 'miss_distance', 'relative_position_r', 'relative_position_t', 'relative_position_n', 'relative_velocity_r',
'relative_velocity_t', 'relative_velocity_n', 'c_weighted_rms', 't_weighted_rms', 'c_sigma_tdot', 't_span', 't_j2k_inc', 'c_j2k_inc', 'c_sigma_r',
'c_crdot_t', 'c_sigma_n', 'c_sigma_t', 'c_sigma_r', 'c_ctdot_t', 'c_position_covariance_det', 't_position_covariance_det']]

In [15]:
grouped_by_event = df_filtered_relevant_columns.groupby('event_id')
for x in range(0,30):
  print(f'Number of CDMs: {x}   Number of events: {(grouped_by_event.size() == x).sum()}')


Number of CDMs: 0   Number of events: 0
Number of CDMs: 1   Number of events: 127
Number of CDMs: 2   Number of events: 441
Number of CDMs: 3   Number of events: 378
Number of CDMs: 4   Number of events: 285
Number of CDMs: 5   Number of events: 264
Number of CDMs: 6   Number of events: 215
Number of CDMs: 7   Number of events: 229
Number of CDMs: 8   Number of events: 195
Number of CDMs: 9   Number of events: 206
Number of CDMs: 10   Number of events: 202
Number of CDMs: 11   Number of events: 217
Number of CDMs: 12   Number of events: 150
Number of CDMs: 13   Number of events: 167
Number of CDMs: 14   Number of events: 182
Number of CDMs: 15   Number of events: 117
Number of CDMs: 16   Number of events: 122
Number of CDMs: 17   Number of events: 200
Number of CDMs: 18   Number of events: 540
Number of CDMs: 19   Number of events: 1856
Number of CDMs: 20   Number of events: 759
Number of CDMs: 21   Number of events: 10
Number of CDMs: 22   Number of events: 0
Number of CDMs: 23   Numb

In [16]:
import numpy as np
# Initialize a list to store the condensed events
condensed_events = []

# Iterate through each group (event)
for event_id, group in grouped_by_event:
    # Skip groups with fewer than 4 CDMs
    if len(group) < 5:
        continue

    # Randomly pick 4 CDMs (sample without replacement) from indices
    random_indices = np.random.choice(group.index[:-1], size=4, replace=False)

    # Select the CDM with the smallest 'time_to_tca' as the fifth CDM
    smallest_tca_cdm = group.loc[group['time_to_tca'].idxmin()]

    # Combine the 4 random CDMs with the smallest TCA CDM
    selected_cdms = pd.concat([group.loc[random_indices], smallest_tca_cdm.to_frame().T])

    # Add to the list of condensed events
    condensed_events.append(selected_cdms)

# Combine all the condensed events into a single DataFrame
condensed_df = pd.concat(condensed_events, ignore_index=True)
display(condensed_df)

,event_id,risk,max_risk_scaling,max_risk_estimate,time_to_tca,mahalanobis_distance,miss_distance,relative_position_r,relative_position_t,relative_position_n,...,t_j2k_inc,c_j2k_inc,c_sigma_r,c_crdot_t,c_sigma_n,c_sigma_t,c_sigma_r,c_ctdot_t,c_position_covariance_det,t_position_covariance_det
0,3.0,-30.000000,137.967248,-7.318307,1.900422,341.106978,23982.0,22.3,10104.3,21749.5,...,98.151369,70.872300,59.302167,-0.999907,73.760653,3800.643367,59.302167,0.206545,3.491587e+13,3.618690e+06
1,3.0,-30.000000,133.195787,-7.314079,2.664311,278.360591,24141.0,23.5,10172.9,21893.5,...,98.151372,70.872300,59.556822,-0.999913,73.763833,3907.749992,59.556822,0.208635,3.694311e+13,8.357174e+06
2,3.0,-9.471212,8.748782,-7.044072,5.169648,125.417601,29202.0,18.2,12300.9,26485.4,...,98.151337,70.872022,114.559941,-0.999998,116.378907,21890.943333,114.559941,0.207144,1.081288e+16,2.618661e+07
3,3.0,-30.000000,120.321521,-7.304081,2.868638,216.960968,24699.0,17.9,10409.1,22398.6,...,98.151367,70.872300,60.380278,-0.999926,73.775016,4236.178703,60.380278,0.215532,4.350700e+13,1.132211e+07
4,3.0,-30.000000,45213.064390,-8.437707,0.656487,458.115245,23864.0,51.9,10055.2,21643.0,...,98.151375,70.871582,43.327947,-0.993422,93.127182,318.178409,43.327947,-0.675979,2.077067e+11,3.502220e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28150,13152.0,-30.000000,1510.933859,-7.111427,1.484446,104.949380,24980.0,-1158.4,-24911.2,1453.7,...,91.931196,98.892527,39.003359,-0.977594,28.249881,500.099090,39.003359,-0.032777,1.799527e+11,1.739638e+03
28151,13152.0,-30.000000,120.059571,-7.016464,5.792345,29.164744,25077.0,-749.6,-25020.6,1517.3,...,91.931194,98.895070,116.984871,-0.998095,153.680025,1629.941717,116.984871,-0.149413,1.837815e+14,1.464684e+05
28152,13152.0,-30.000000,116.922865,-7.009439,5.487888,29.132810,24956.0,-748.2,-24899.2,1509.6,...,91.931194,98.895070,117.020340,-0.998132,153.680318,1645.923145,117.020340,-0.147092,1.875652e+14,1.146651e+05
28153,13152.0,-30.000000,120.768538,-7.018091,5.140574,29.508442,25114.0,-748.8,-25057.5,1519.6,...,91.931191,98.895070,116.970937,-0.998092,153.679895,1628.472904,116.970937,-0.149356,1.834979e+14,1.070936e+05


The code below will separate the target and feature columns and store them in 1 large array for training purposes.

In [19]:
# Include `event_id` as a feature
features = []
targets= []

# Iterate through each event
for event_id, group in condensed_df.groupby('event_id'):
    # Include `event_id` in the features
    feature = group.drop(columns=['risk']).values  # Only exclude the target column
    target = group['risk'].values  # Extract the target column

    # Append to lists
    features.append(feature)
    targets.append(target)

# Display the updated feature and target extraction
len(features), len(targets), features[:1], targets[:1]


(5631,
 5631,
 [array([[ 3.00000000e+00,  1.37967248e+02, -7.31830661e+00,
           1.90042209e+00,  3.41106978e+02,  2.39820000e+04,
           2.23000000e+01,  1.01043000e+04,  2.17495000e+04,
          -3.97000000e+01, -1.23109000e+04,  5.71890000e+03,
           2.48400000e+00,  1.13200000e+00,  5.69049646e-02,
           1.50000000e+00,  9.81513694e+01,  7.08723003e+01,
           5.93021669e+01, -9.99907375e-01,  7.37606535e+01,
           3.80064337e+03,  5.93021669e+01,  2.06545100e-01,
           3.49158715e+13,  3.61868976e+06],
         [ 3.00000000e+00,  1.33195787e+02, -7.31407921e+00,
           2.66431059e+00,  2.78360591e+02,  2.41410000e+04,
           2.35000000e+01,  1.01729000e+04,  2.18935000e+04,
          -3.98000000e+01, -1.23109000e+04,  5.71890000e+03,
           2.48400000e+00,  1.11800000e+00,  5.69378082e-02,
           1.50000000e+00,  9.81513717e+01,  7.08723003e+01,
           5.95568216e+01, -9.99912896e-01,  7.37638326e+01,
           3.90774999e+03,